<a href="https://colab.research.google.com/github/pragatheeswaran-cds/IISC-CAPSTONE-PROJECT/blob/praga-upload/Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### Budget

In [ ]:
df = pd.read_csv('Budget.csv')
df.shape

In [ ]:
df['Category'].unique()

In [ ]:
ax = df.plot(kind='bar', x='Category', title="Budget", ylabel='Amount in USD', grid=True)
ax.bar_label(ax.containers[0])
plt.show()

### Personal Transactions

In [ ]:
df_fin = pd.read_csv('personal_transactions.csv', parse_dates=['Date'])
df_fin.shape

In [ ]:
df_fin.columns

In [ ]:
df_fin.info()

In [ ]:
df_fin['Account Name'].unique()

In [ ]:
account_count = df_fin['Account Name'].value_counts()

In [ ]:
plt.pie(account_count, labels=account_count.index, autopct='%1.1f%%', startangle=90)
plt.title("Type of Account Usage")
plt.tight_layout()
plt.show()

In [ ]:
df_fin.Category.unique()

In [ ]:
df_fin[df_fin.duplicated]

In [ ]:
df_fin['Category'] = df_fin['Category'].replace('Food & Dining', 'Fast Food')

In [ ]:
df_fin['Date'].min(), df_fin['Date'].max()

In [ ]:
df_fin['month'] = df_fin['Date'].dt.month

In [ ]:
df_fin['year'] = df_fin['Date'].dt.year

In [ ]:
df_fin[df_fin['Amount'] <= 0]

In [ ]:
df_fin['Transaction Type'].unique()

In [ ]:
df_fin[df_fin['Transaction Type'] == 'credit']['Category'].unique()

In [ ]:
df_fin['Transaction Type'] = np.where(
    (df_fin['Category'] == 'Credit Card Payment') & (df_fin['Transaction Type'] == 'credit'),
    'debit',
    df_fin['Transaction Type']
)

In [ ]:
df_fin[df_fin['Transaction Type'] == 'credit']['Category'].unique()

In [ ]:
#TODO check some months have higher paycheck

In [ ]:
credit_data = df_fin[df_fin['Transaction Type'] == 'credit'].groupby(['year','month']).sum('Amount')
debit_data = df_fin[df_fin['Transaction Type'] == 'debit'].groupby(['year','month']).sum('Amount')

In [ ]:
credit_data.reset_index(inplace=True)
debit_data.reset_index(inplace=True)

credit_data['date'] = pd.to_datetime(credit_data[['year', 'month']].assign(day=1))
debit_data['date'] = pd.to_datetime(debit_data[['year', 'month']].assign(day=1))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(credit_data['date'], credit_data['Amount'], label = 'Income', marker='o')
plt.plot(debit_data['date'], debit_data['Amount'], label = 'Expense', marker='o', color='r')
plt.title('Monthly Income vs Expenses')
plt.xlabel('Month')
plt.ylabel('Amount ($)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
df_fin[(df_fin['Transaction Type'] == 'debit') & (df_fin['month'] == 10) & (df_fin['year'] == 2018)]

### Alpha Vantage

In [ ]:
import requests

In [ ]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=1Y92HFSN1KP5GN0Q&outputsize=full'
r = requests.get(url)
data = r.json()

print(data)

In [ ]:
# Your data is already a dictionary with nested structure
# Extract the time series data
time_series = data['Time Series (Daily)']

# Convert to DataFrame
stock_data = pd.DataFrame.from_dict(time_series, orient='index')

# Convert string columns to numeric
for col in stock_data.columns:
    stock_data[col] = pd.to_numeric(stock_data[col])

# Rename columns for easier access
stock_data.columns = ['open', 'high', 'low', 'close', 'volume']

# Convert index to datetime
stock_data.index = pd.to_datetime(stock_data.index)

# Sort by date (oldest to newest)
stock_data = stock_data.sort_index()

print(stock_data.shape)
print(stock_data.head())

In [ ]:
stock_data.shape

In [ ]:
start_date = '2022-01-01'
stock_data_3y = stock_data[stock_data.index >= start_date]

In [ ]:
stock_data_3y.shape

In [ ]:
df= stock_data_3y.copy()

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

In [ ]:
# Plot the closing values for Microsoft
plt.figure(figsize=(17,8))
plt.plot(df['close'])
plt.title('IBM Closing Values')
plt.xlabel('Dates')
plt.ylabel('Close')
plt.legend()

In [ ]:
df.tail(10)

In [ ]:
df.index = pd.to_datetime(df.index)
full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
df_full = df.reindex(full_range)

df_full[['open','high','low','close']] = df_full[['open','high','low','close']].ffill()  # forward fill for prices
df_full['volume'] = df_full['volume'].fillna(0)  # no trades on holidays


df_full.index.name = 'date'
print(df_full.tail(10))

In [ ]:
df_weekly = df_full.resample('W').mean()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(df_weekly['close'], model='multiplicative')

In [ ]:
result.plot()
plt.tight_layout()
plt.xticks(rotation=45)
plt.show()

#### Weekly

In [ ]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY_ADJUSTED&symbol=IBM&apikey=1Y92HFSN1KP5GN0Q&outputsize=full'
r = requests.get(url)
data = r.json()

print(data)

In [ ]:
time_series = data['Weekly Adjusted Time Series']

In [ ]:
stock_data = pd.DataFrame.from_dict(time_series, orient='index')

In [ ]:
for col in stock_data.columns:
    stock_data[col] = pd.to_numeric(stock_data[col])

In [ ]:
stock_data.columns = ['open', 'high', 'low', 'close', 'adjusted_close', 'volume', 'dividend_amount']

In [ ]:
stock_data.index = pd.to_datetime(stock_data.index)

In [ ]:
stock_data = stock_data.sort_index()

In [ ]:
# Calculate daily returns
stock_data['returns'] = stock_data['adjusted_close'].pct_change()

# Annualized return (mean * trading days)
annual_return = stock_data['returns'].mean() * 252

# Annualized volatility (std * sqrt(trading days))
annual_volatility = stock_data['returns'].std() * (252 ** 0.5)

print(f"Annual Return: {annual_return:.2%}")
print(f"Annual Volatility: {annual_volatility:.2%}")


In [ ]:
def classify_stock(ret, vol):
    if ret >= 0.15 and vol <= 0.20:
        return "High Return / Low Risk"
    elif ret >= 0.15 and vol > 0.20:
        return "High Return / High Risk"
    elif ret < 0.15 and vol <= 0.20:
        return "Low Return / Low Risk"
    else:
        return "Low Return / High Risk"

category = classify_stock(annual_return, annual_volatility)
print("Category:", category)


In [ ]:
returns = stock_data['returns'].dropna()

In [ ]:
plt.figure(figsize=(9,4))
sns.kdeplot(returns * 100, color='brown', linewidth=2)
plt.xlabel("Return, %")
plt.ylabel("Probability")
plt.title("IBM (Empirical Return Distribution)")



### Insurance Dataset

In [ ]:
ins_data = pd.read_csv('insurance_dataset.csv')
ins_data.shape

In [ ]:
ins_data.head()

In [ ]:
ins_data.info()

In [ ]:
ins_data.medical_history.unique()

In [ ]:
ins_data.family_medical_history.unique()

In [ ]:
ins_data['medical_history'] = ins_data['medical_history'].fillna('No Record')
ins_data['family_medical_history'] = ins_data['family_medical_history'].fillna('No Record')

In [ ]:
ins_data.describe()

In [ ]:
import seaborn as sns
sns.boxplot(ins_data[['age','bmi']])

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(data=ins_data, x="age", kde=True, bins=48)
plt.title("Age Distribution")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(data=ins_data, x="bmi", kde=True, bins=35)
plt.title("BMI Distribution")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(data=ins_data, x="charges", kde=True, bins=35)
plt.title("Charges ($) Distribution")
plt.tight_layout()
plt.show()

In [ ]:
smoker_labels = ins_data['smoker'].map(lambda x: 'Smoker' if x == 'yes' else 'Non-Smoker')
smoker_counts = smoker_labels.value_counts()
plt.pie(smoker_counts, labels=smoker_counts.index, autopct='%1.1f%%', startangle=90)
plt.title("Smoker vs Non-Smoker")
plt.tight_layout()
plt.show()

In [ ]:
smoker_counts

In [ ]:
fig = sns.histplot(ins_data,
                  x='charges',
                  hue='smoker'
)
plt.title('Charges vs. Smoker')

In [ ]:
sns.histplot(data=ins_data,x='charges',hue='coverage_level', kde=True)

In [ ]:
sns.histplot(data=ins_data,x='charges',hue='region', kde=True)

In [ ]:
sns.histplot(data=ins_data,x='charges',hue='exercise_frequency', kde=True)

In [ ]:
ins_data[ins_data.duplicated]

In [ ]:
import pandas as pd

df_encoded = pd.get_dummies(ins_data, columns=['smoker', 'region', 'gender', 'medical_history','family_medical_history','exercise_frequency','occupation','coverage_level'], drop_first=False)

In [ ]:
cols = [c for c in df_encoded.columns if c != 'charges'] + ['charges']
df_encoded = df_encoded[cols]

In [ ]:
plt.figure(figsize=(30,25))
sns.heatmap(df_encoded.corr(), cmap='coolwarm', annot=True, fmt='0.2f' )
plt.title("Correlation Heatmap")
plt.tight_layout()
plt.show()